In [ ]:
#-- Install ultralytics for YOLO  --------------------------------------------------------------------------------
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Import -----------------------------------------------------------------------------------------------
from ultralytics import YOLO
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import csv
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Initialize ---------------------------------------------------------------------------------------------------
intput_path = '/kaggle/input/'
out_path = '/kaggle/working/'

result_video_dir = out_path + 'result_videos/'
CONF_THRESHOLD = 0.2
IOU_THRESHOLD = 0.5

BRUGLARY_THRESHOLD = 5

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:' , DEVICE)
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Set labels for ZSOD Models ------------------------------------------------------------------------------------
labels = [
    "Person climbing over a fence",
    "Person climbing a wall",
    "Person breaking a lock with tools",
    "Person trying to pick a lock",
    "Person forcing a door open with strength",
    "Person peeking through a window",
    #"Person carrying stolen items",
    "Person sneaking into a building",
    "Person looking around nervously",
    "Person hiding behind an object",
    "Person running away from a building",
    "Person carrying tools like a crowbar",
    "Person breaking a window with an object",
    "Person tampering with a security camera",
    "Person cutting alarm wires",
    #"Person jumping over a barrier",
    #"Person carrying a large bag suspiciously",
    "Person entering a restricted area",
    #"Person hiding stolen items in a bag",
    # "Person fighting with a security guard",
    #"Person avoiding eye contact with others",
    "Person loitering near a building",
    "Person jumping out of a window",
    "Person disabling an alarm system",
    "Person wearing a mask and avoiding detection"
] 
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
#-- Create and Initialize Model ----------------------------------------------------------------------------------
#-- YOLO World --
model_yolo_world_zsod = YOLO('yolov8x-worldv2.pt')
model_yolo_world_zsod.set_classes(labels)

display.clear_output()
print('YOLO model was loaded successfully :)')
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
burglary_samples_dir = '/kaggle/input/anomalydetectiondatasetucf/Burglary/'
normal_samples_dir = '/kaggle/input/anomalydetectiondatasetucf/Normal_Videos_for_Event_Recognition/Normal_Videos_for_Event_Recognition/'

In [ ]:
detected_index_labels = {}
detected_name_labels = {}

for video_file in os.listdir(burglary_samples_dir):    
    #-- log --
    print(f'Processing {video_file} ==========================================================') 

    if video_file != 'Burglary081_x264A.mp4':
        continue
    
    project_value = 'result_videos'
    dot_index = video_file.rfind('.') 
    name_value = video_file[:dot_index]
    
    video_path = os.path.join(burglary_samples_dir, video_file)       

    # Calculate frame rate (fps) using OpenCV
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    
    BRUGLARY_THRESHOLD = 1 * fps  # Threshold is 5 times the frame rate
    print(f'BRUGLARY_THRESHOLD: {BRUGLARY_THRESHOLD} - fps: {fps}')

    index_labels_count = {}
    name_labels_set = set()

    i = 1
    
    for r in model_yolo_world_zsod.predict(source = video_path,
                                           conf = CONF_THRESHOLD,
                                           iou = IOU_THRESHOLD,
                                           show = False,
                                           save= True,
                                           stream = True,
                                           project= project_value,
                                           name=name_value) :        
        #index_labels_set.update(set(r.boxes.cls.int().tolist()))
        for cls_index in r.boxes.cls.int().tolist():
            if cls_index in index_labels_count:
                index_labels_count[cls_index] += 1
            else:
                index_labels_count[cls_index] = 1

        # print(r.boxes)
        
        

        i += 1
        if i>=700:
            break
        pass

    for index, count in index_labels_count.items():
        if count > BRUGLARY_THRESHOLD:
            name_labels_set.add(labels[index])
        
    detected_index_labels[name_value] = name_labels_set
    
    break
    
display.clear_output()
print(':)')

for video, lbls in detected_index_labels.items():
    print(f'{video}: ----------------------------------\n{lbls}')

for lbl, num in index_labels_count.items():
    print(f'{lbl}: {labels[lbl]} : {num}')

In [ ]:
# with open("burglary.csv", "w", newline='') as file:
#     writer = csv.writer(file)
#     for key, value in detected_index_labels.items():
#         writer.writerow([key, value])

# print("Dictionary saved to my_dict.csv")

In [ ]:
detected_index_labels = {}
detected_name_labels = {}

for video_file in os.listdir(normal_samples_dir):    
    #-- log --
    print(f'Processing {video_file} ==========================================================') 

    if video_file != 'Normal_Videos_150_x264.mp4':
        continue
    
    project_value = 'result_videos'
    dot_index = video_file.rfind('.') 
    name_value = video_file[:dot_index]
    
    video_path = os.path.join(normal_samples_dir, video_file)

    # Calculate frame rate (fps) using OpenCV
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    cap.release()
    
    BRUGLARY_THRESHOLD = 1 * fps  # Threshold is 5 times the frame rate
    print(f'BRUGLARY_THRESHOLD: {BRUGLARY_THRESHOLD} - fps: {fps}')

    index_labels_count = {}
    name_labels_set = set()

    i = 1
    
    for r in model_yolo_world_zsod.predict(source = video_path,
                                           conf = CONF_THRESHOLD,
                                           iou = IOU_THRESHOLD,
                                           show = False,
                                           save= True,
                                           stream = True,
                                           project= project_value,
                                           name=name_value) :        
        #index_labels_set.update(set(r.boxes.cls.int().tolist()))
        for cls_index in r.boxes.cls.int().tolist():
            if cls_index in index_labels_count:
                index_labels_count[cls_index] += 1
            else:
                index_labels_count[cls_index] = 1

        # print(r.boxes)
        
        

        i += 1
        if i>=300:
            break
        pass

    for index, count in index_labels_count.items():
        if count > BRUGLARY_THRESHOLD:
            name_labels_set.add(labels[index])
        
    detected_index_labels[name_value] = name_labels_set
    
    break
    
display.clear_output()
print(':)')

for video, lbls in detected_index_labels.items():
    print(f'{video}: ----------------------------------\n{lbls}')

for lbl, num in index_labels_count.items():
    print(f'{lbl}: {labels[lbl]} : {num}')

In [ ]:
# with open("normal.csv", "w", newline='') as file:
#     writer = csv.writer(file)
#     for key, value in detected_index_labels.items():
#         writer.writerow([key, value])

# print("Dictionary saved to normal.csv")